In [72]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import os

np.random.seed(42)
tf.random.set_seed(42)

In [49]:
# Definir o caminho para o diretório contendo as imagens de treino e teste
data_dir = 'dataset/Kather_texture_2016_image_tiles_5000'
samples, labels = list(), list()

for nome_pasta in os.listdir(data_dir):
    for nome_arquivo in os.listdir(os.path.join(data_dir, nome_pasta)):
        samples.append(os.path.join(nome_pasta,nome_arquivo))
        labels.append(int(nome_pasta[:2]))



In [50]:
all_data = pd.DataFrame({'path':samples,'label':labels})
all_data['label'] = all_data['label'].astype('str')

In [65]:
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.9,  # 20% dos dados serão usados para validação
                             #horizontal_flip=True,
                             )

In [75]:
# Configuração dos parâmetros da CNN
input_shape = (28, 28, 3)  # Dimensões da imagem de entrada
num_classes = 8  # Número de classes no seu conjunto de dados
batch_size = 32

# Criando os geradores de dados de treinamento e validação usando flow_from_dataframe
train_generator = datagen.flow_from_dataframe(dataframe=all_data,
                                              directory=data_dir,
                                              x_col="path",
                                              y_col="label",
                                              subset="training",  # Usando os dados de treinamento
                                              batch_size=32,
                                              target_size=(28, 28),  # Tamanho da imagem para o modelo MobileNetV2
                                              class_mode="categorical")  # Se for um problema de classificação multiclasse

valid_generator = datagen.flow_from_dataframe(dataframe=all_data,
                                              directory=data_dir,
                                              x_col="path",
                                              y_col="label",
                                              subset="validation",  # Usando os dados de validação
                                              batch_size=32,
                                              target_size=(28, 28),
                                              class_mode="categorical")


Found 500 validated image filenames belonging to 8 classes.
Found 4500 validated image filenames belonging to 8 classes.


In [76]:
#base_model = EfficientNetB0(input_shape=(32, 32, 3), include_top=False, weights="imagenet")
# for layer in base_model.layers:
#     layer.trainable = False
# model = models.Sequential([
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.3),
#     layers.Dense(num_classes, activation='softmax')  # Substitua 'num_classes' pelo número de classes do seu problema
# ])

In [79]:
# Criando o modelo CNN
model = Sequential()

# Camadas convolucionais e de pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten para conectar as camadas totalmente conectadas
model.add(Flatten())

# Camadas totalmente conectadas
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Substitua 'num_classes' pelo número de classes do seu problema

# Compilando o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Visualizando a arquitetura do modelo
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 1, 1, 128)        

In [80]:
# Treinando o modelo
model.fit(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
16/16 [==============================] - 3s 139ms/step - loss: 0.8613 - accuracy: 0.8700 - val_loss: 16.0334 - val_accuracy: 0.0278
Epoch 2/10
16/16 [==============================] - 2s 126ms/step - loss: 5.4612e-06 - accuracy: 1.0000 - val_loss: 46.3024 - val_accuracy: 0.0278
Epoch 3/10
16/16 [==============================] - 2s 124ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 57.1185 - val_accuracy: 0.0278
Epoch 4/10
16/16 [==============================] - 2s 124ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 59.7294 - val_accuracy: 0.0278
Epoch 5/10
16/16 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 60.2927 - val_accuracy: 0.0278
Epoch 6/10
16/16 [==============================] - 2s 124ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 60.4093 - val_accuracy: 0.0278
Epoch 7/10
16/16 [==============================] - 2s 125ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 60.433